# Quantum portfolio optimization (Qiskit)

We will try to solve the same problem using Qiskits Finance module. As a remainder:

We have a number of assets $n$ to chose from with a budget $b$, smaller than $n$ of course ($n \gt b$). This asset selection should retrieve the biggest outcome according to individual weigths ($\mu$) and the factor of the combination between two assets (two by two in our case) interpreted as the risk. We must think of our solution as a binary vector $X$ of length $n$. An additional risk factor on the inversion could be added ($q$) conditioning the uncertainty of those weigths.

Optimization problem:
$$
\max \sum_i \mu_ix_i - q \sum_i \sum_j \sigma_{i,j}x_ix_j \\
s.t. \sum_i x_i \lt b
$$

We will also explore how this problem can be formulated as a QUBO problem. Let us choose a set of assets to evaluate.

In [1]:
from quanvia.finance import utils

# Get first 25 elements of USDT list
asset_list = utils.get_binance_assets()
asset_list = [x for x in asset_list if x.endswith("USDT")]
asset_list = asset_list[:25]

# Get the data
df = utils.get_binance_data(asset_list)
mu, sigma = utils.get_exp_cov(df)
mu_vals = list(mu.values())

Thanks to Qiskit's own functionalities we can easilly obtain our QUBO problem.

In [2]:
from qiskit_finance.applications.optimization import PortfolioOptimization

# Variables
num_assets = len(mu) # not all 25 assets may have the whole year informed
q = 0.5  # set risk factor
budget = 2  # set budget

# Problem definition
portfolio = PortfolioOptimization(expected_returns=mu_vals, covariances=sigma, risk_factor=q, budget=budget)
qp = portfolio.to_quadratic_program()
qp

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Portfolio optimization

Minimize
 obj: - 0.001001034692 x_0 - 0.004088714376 x_1 - 0.009140994591 x_2
      - 0.002936158866 x_3 - 0.001184433654 x_4 - 0.006641321952 x_5
      - 0.005959966036 x_6 - 0.005231292502 x_7 - 0.002716377955 x_8
      + 0.000002074018 x_9 - 0.005898171205 x_10 - 0.001400326839 x_11
      - 0.003122088567 x_12 - 0.004232123703 x_13 - 0.006943245748 x_14
      - 0.004569600516 x_15 - 0.006212286509 x_16 - 0.005777508401 x_17
      - 0.000001742906 x_18 - 0.003866432305 x_19 + [ 0.001742073407 x_0^2
      + 0.003597404091 x_0*x_1 + 0.003983593072 x_0*x_2 + 0.004136077679 x_0*x_3
      + 0.004009847314 x_0*x_4 + 0.004359444330 x_0*x_5 + 0.003260227921 x_0*x_6
      + 0.003673372545 x_0*x_7 + 0.004332512926 x_0*x_8 - 0.000002488239 x_0*x_9
      + 0.004178310482 x_0*x_10 + 0.003657489095 x_0*x_11
      + 0.004236256648 x_0*x_12 + 0.003694792609 x_0*x_13
      + 0.003799709987 x_0*x_14 

What we see is the minimization versión of the above problem on the more common $\min x^TQx$ shape. This allows us to translate the $Q$ matrix into our problem Hamiltonian.

A simple way to solve this problem classically is to use *NumpyMinimumEigenSolver* function.

In [3]:
%%time

from qiskit.utils import algorithm_globals
from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit_optimization.algorithms import MinimumEigenOptimizer

# This parameter is needed for large matrixes
algorithm_globals.massive = True
exact_mes = NumPyMinimumEigensolver()
exact_eigensolver = MinimumEigenOptimizer(exact_mes)

# Solve
result = exact_eigensolver.solve(qp)
print(result)

optimal function value: -0.0071206523339103905
optimal value: [0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
status: SUCCESS
Wall time: 8min 19s


Well, not as efficient as expected. We could try by using the available *CplexOptimizer* instead.

In [9]:
%%time

from qiskit_optimization.algorithms import CplexOptimizer

cplex_optimizer = CplexOptimizer()

# Solve
result = cplex_optimizer.solve(qp)
print(result)

optimal function value: -0.00606055123258371
optimal value: [0. 1. 1. 0. 0.]
status: SUCCESS
Wall time: 67.1 ms


Big difference indeed! But of course, this is not quantum computing... for that some quantum hardware must be used. Thanks to IBM accesible backend we can make use of them. First, we could just try to solve our problem by means of Variational Quantum Eigensolver algorithm.

In [5]:
from qiskit import IBMQ

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_bogota') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_extended_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_m

Plenty to choose from! Let us pickup the least busy one but we need to make sure its dimensions match the ones required by our problem (20 variables shown above).

In [6]:
devices = provider.backends(filters=lambda x: x.configuration().n_qubits >= len(mu)
                                    and not x.configuration().simulator
                                    and x.status().operational==True)
len(devices)

0

Ups! There is no IBM Hardware available that will be able to run our problem. So far lets try with fewer assets and simulator oprtions.

In [7]:
num_assets = 6
asset_list = utils.get_binance_assets()
asset_list = [x for x in asset_list if x.endswith("USDT")]
asset_list = asset_list[:num_assets]

# Get the data
df = utils.get_binance_data(asset_list)
mu, sigma = utils.get_exp_cov(df)
mu_vals = list(mu.values())

# Variables
num_assets = len(mu) # not all assets may have the whole year informed
q = 0.5  # set risk factor
budget = 2  # set budget

# Problem definition
portfolio = PortfolioOptimization(expected_returns=mu_vals, covariances=sigma, risk_factor=q, budget=budget)
qp = portfolio.to_quadratic_program()

In [8]:
from qiskit.providers.ibmq import least_busy

devices = provider.backends(filters=lambda x: x.configuration().n_qubits >= num_assets and not x.configuration().simulator)
backend = least_busy(devices)
backend.name()

'ibmq_bogota'

In [ ]:
%%time

from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import SLSQP

optimizer = SLSQP(maxiter = 90000)  
vqe = VQE(optimizer=optimizer, quantum_instance=backend)

vqe_meo = MinimumEigenOptimizer(vqe)
result = vqe_meo.solve(qp)

print(result)  

Given its hybrid nature, each job is set to the backend service to be executed and the optimizer runs locally tunning the parameters of the variational quantum circuit for the next run until it converges to the expected result. This network times and queue time previous to gaining access to the hardware will ipact the overall performance.

![vqe](../assets/vqe1iter.PNG)

In the same way, Quantum Approximation Optimization Algorithm (QAOA) is a digitazed version inspired by quantum annealers iterating by pairs of Cost and Mixed Hamiltonians to our objective.

In [ ]:
%%time

from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import COBYLA
from qiskit.utils import QuantumInstance

# Random seed
seed = 123

cobyla = COBYLA()
cobyla.set_options(maxiter=250)
quantum_instance = QuantumInstance(backend=backend, seed_simulator=seed, seed_transpiler=seed)
qaoa_mes = QAOA(optimizer=cobyla, reps=3, quantum_instance=quantum_instance)
qaoa = MinimumEigenOptimizer(qaoa_mes)
result = qaoa.solve(qp)

print(result)  

Due to this, IBM released its Runtimes. They should minimize the network time of our total execution.

In [11]:
devices = provider.backends(filters=lambda x: x.configuration().n_qubits >= num_assets)
backend = least_busy(devices)
backend.name()

'ibmq_qasm_simulator'

In [ ]:
from qiskit.opflow import I, Z
from qiskit.algorithms.optimizers import COBYLA
from qiskit_optimization.runtime import QAOAClient

# define diagonal Hamiltonian whose minimum eigenvalue we want to find as PauliOps
op =  (Z ^ Z ^ I ^ I ^ I) - (I ^ I ^ Z ^ Z ^ I) 

# set up the client and solve the problem
client = QAOAClient(
    reps=2,  # use p=2 repetitions in the QAOA ansatz
    optimizer=COBYLA(), 
    alpha=0.75,  # use CVaR expectation with 75% of the best readouts
    provider=provider, 
    backend=backend
)
result = client.compute_minimum_eigenvalue(op)